In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, MaxPooling1D, Conv1D, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
from transformer import TransformerBlock, TokenAndPositionEmbedding
from utils import load_dataset, train, plot_roc, auc
from einops import rearrange

In [2]:
maxlen = 1000
sample_size = 4
embed_dim = 64  # Embedding size for each token
num_heads = 64  # Number of attention heads
ff_dim = 1024  # Hidden layer size in feed forward network inside transformer

clear_session()

inputs = Input(shape=(1000, 4))
x = Conv1D(embed_dim, kernel_size=26, activation="relu")(inputs)
x = MaxPooling1D(pool_size=13, strides=13)(x)
x = Dropout(0.2)(x)
# x = TokenAndPositionEmbedding(maxlen, sample_size, embed_dim)(x)
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
x = Flatten()(x)
x = Dense(925, activation="relu")(x)
outputs = Dense(1, activation="relu")(x)

model = Model(inputs, outputs, name="conv_transformer")

model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "conv_transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 4)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 975, 64)           6720      
____________________________________________

In [32]:
es = EarlyStopping(monitor="val_loss", patience=15)
optimizer = Adam(lr=1e-4)

epochs = 1000
validation_freq = 1
n = 3
file = f"MA0035_{n}000"
boundary = n * 1000

In [13]:
X_train, y_train, X_test, y_test = load_dataset(file=file, labels="binlabels")
X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

# np.where(y_train > boundary, 0, 1)
# np.where(y_test > boundary, 0, 1)

In [33]:
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"])

In [34]:
model = train(
    dataset=(X_train, y_train, X_test, y_test),
    model=model,
    epochs=epochs,
    validation_freq=validation_freq,
    callbacks=[es],
    verbose=1,
    batch_size=100
)

y_pred = model.predict(X_test)
plot_roc(y_test, y_pred)
print(auc(y_test, y_pred))

Epoch 1/1000
 473/1000 [=============>................] - ETA: 12s - loss: 7.5745 - acc: 0.5021

KeyboardInterrupt: 

model.evaluate(X_test, y_test)

model.save('../model_saves/transformer-model.h5')